In [4]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 5GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/sound-model/stage-sound2.pth
/kaggle/input/sound-test3/Playlist_Test_Spectogram/Amir_spectogram/fwpaxgrz.png
/kaggle/input/sound-test3/Playlist_Test_Spectogram/Amir_spectogram/fsbjgveg.png
/kaggle/input/sound-test3/Playlist_Test_Spectogram/Amir_spectogram/hdshjhuf.png
/kaggle/input/sound-test3/Playlist_Test_Spectogram/Amir_spectogram/gnmqmwmx.png
/kaggle/input/sound-test3/Playlist_Test_Spectogram/Amir_spectogram/adosednd.png
/kaggle/input/sound-test3/Playlist_Test_Spectogram/Amir_spectogram/crpdavon.png
/kaggle/input/sound-test3/Playlist_Test_Spectogram/Akshay_spectogram/usgrcggb.png
/kaggle/input/sound-test3/Playlist_Test_Spectogram/Akshay_spectogram/aqsftlag.png
/kaggle/input/sound-test3/Playlist_Test_Spectogram/Akshay_spectogram/alcgsfbi.png
/kaggle/input/sound-test3/Playlist_Test_Spectogram/Akshay_spectogram/nrfrftpi.png
/kaggle/input/sound-test3/Playlist_Test_Spectogram/Akshay_spectogram/llexivju.png
/kaggle/input/sound-test3/Playlist_Test_Spectogram/Akshay_spectogra

In [5]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline

In [6]:
from fastai.vision import *
from fastai.metrics import error_rate
import fastai

In [7]:
# bs = 64
bs = 8   

In [8]:
import os
print(os.listdir("../input/"))

['sound-model', 'sound-test3', 'spectogramvalidate', 'spectogram-train', 'sound-model-unfreezed', 'sound-train-extended2']


In [9]:
import os
print(os.listdir("/"))

['bin', 'var', 'mnt', 'etc', 'root', 'home', 'tmp', 'lib', 'sbin', 'proc', 'lib64', 'srv', 'boot', 'opt', 'sys', 'media', 'usr', 'run', 'dev', 'kaggle', '.dockerenv', '.jupyter', 'src', 'run_jupyter.sh', 'entrypoint.sh']


In [12]:
import os
print(os.listdir("../input/sound-train-extended2/Playlist_Extended_Spectogram/"))

['Amir_spectogram', 'Akshay_spectogram', 'Salman_spectogram', 'Modi_spectogram', 'Srk_spectogram']


In [ ]:
print(os.listdir("../input/sound-train-extended2/Playlist_Extended_Spectogram/"))

In [1]:
!ls "../input/spectogram-train/"




ls: ../input/spectogram-train/: No such file or directory


In [2]:
import pathlib

In [3]:
path = pathlib.Path('../input/sound-train-extended2/')

In [14]:
path_img = path/'Playlist_Extended_Spectogram/'/'Akshay_spectogram'

In [15]:
doc(ImageDataBunch.from_folder)

In [16]:
# path_anno = path/'annotations'
data = ImageDataBunch.from_folder(path,train = 'Playlist_Extended_Spectogram',ds_tfms=get_transforms(), size=224, bs=bs,valid_pct=0.20
                                  ).normalize(imagenet_stats)
data.classes

['Akshay_spectogram',
 'Amir_spectogram',
 'Modi_spectogram',
 'Salman_spectogram',
 'Srk_spectogram']

In [17]:
fnames = get_image_files(path_img)
fnames[:5]

[PosixPath('../input/sound-train-extended2/Playlist_Extended_Spectogram/Akshay_spectogram/hlyexuyt.png'),
 PosixPath('../input/sound-train-extended2/Playlist_Extended_Spectogram/Akshay_spectogram/knqoodtf.png'),
 PosixPath('../input/sound-train-extended2/Playlist_Extended_Spectogram/Akshay_spectogram/zexdutqm.png'),
 PosixPath('../input/sound-train-extended2/Playlist_Extended_Spectogram/Akshay_spectogram/ccdrrwcc.png'),
 PosixPath('../input/sound-train-extended2/Playlist_Extended_Spectogram/Akshay_spectogram/zhyiqpqn.png')]

In [ ]:
data.show_batch(rows=4, figsize=(7,6))

In [ ]:
print(data.classes)
len(data.classes),data.c

## Training: resnet34

In [ ]:
learn = cnn_learner(data, models.resnet34, metrics=[accuracy,error_rate],model_dir="/kaggle/working/model/")

In [ ]:
learn.model

In [ ]:
learn.load('stage-sound-unfreezed');

In [ ]:
learn.unfreeze()

In [ ]:
learn.fit_one_cycle(1)

In [ ]:
learn.lr_find()

In [ ]:
learn.recorder.plot()

In [ ]:

learn.fit_one_cycle(50, max_lr=slice(0.5e-4,0.5e-2))

In [ ]:
saved_path = learn.save('stage-sound-unfreezed',return_path=True)
saved_path

In [ ]:
doc(load_learner)

In [ ]:
fastai.__version__

In [ ]:
doc(open_image)

In [ ]:
!ls ../input/spectogramvalidate/Playlist_old_Spectogram


In [ ]:
doc(ClassificationInterpretation.from_learner)

In [ ]:
interp = ClassificationInterpretation.from_learner(learn)
interp.plot_confusion_matrix(figsize=(12,12), dpi=60)

In [ ]:
root = pathlib.Path('../input/sound-test3/Playlist_Test_Spectogram/')

In [ ]:
for path, subdirs, files in os.walk(root):
        for name in files:
            print(path)
#             print(name)
            path_img = os.path.join(path, name)
            img_test = open_image(path_img)
#             print(path_img)
            res = learn.predict(img_test)
            print(res[0],res[1])
            for index,v in enumerate(res[2]):
                print(data.classes[index],v)
            print('\n')